In [ ]:
import pandas as pd
import datetime as dt
from bs4 import BeautifulSoup
import requests

In [ ]:
def webscrap_nfl_injury_reports(Start_Year,End_Year):
    #list of the NFL teams for url purposes
    teams = ['crd', 'atl', 'rav', 'buf', 'car', 'chi', 'cin', 'cle', 'dal', 'den', 'det', 'gnb','htx','clt','jax','kan',
             'sdg','ram','mia','min','nor','nwe','nyg','nyj','rai','phi','pit','sea','sfo','tam','oti','was']
    years = []

    #list of years to pull data for url purposes
    for yr in range(Start_Year,End_Year+1):
        years.append(yr)

    #starting points to iterate through
    team = 0
    year = 0
    dfname = []
    
    while team < 1:
        dfname.append(f'{teams[team]}_{years[year]}_injuryreport.csv')
        
        #url for web scraping
        url = f'https://www.pro-football-reference.com/teams/{teams[team]}/{years[year]}_injuries.htm'

        #opening website
        r = requests.get(url)
        soup = BeautifulSoup(r.content, 'lxml')

        #finding table
        table = soup.find('table', attrs={'class': 'sortable', 'id': 'team_injuries'})
        table_rows = table.find_all('tr')

        #scraping the data
        final_data = []
        for tr in table_rows:
            td = tr.find_all(['th','td'])
            row = [tr['data-tip'] if tr.has_attr("data-tip") else tr.text for tr in td]
            final_data.append(row)

        # update of location of web scrape
        print(teams[team],years[year])

        #creatingdataframe to save
        dfdata = final_data[1:]
        data_body = [[dfdata[j][i] for j in range(len(dfdata))] for i in range(len(dfdata[0]))]
        data = {key: pd.DataFrame(data_body,final_data[0]).T for key in dfname}

#             while year < End_Year+1:
#                 for key in data:
#                     data[key].insert(loc=3,column='Year',value=year,allow_duplicates=True)
#                     data[key]['Year'] = data[key]['Year'].astype(int)
#                     year += 0.5

#             for key in data:
#                 data[key].drop(['TTL','AVG'],axis=1,inplace=True)
#                 data[key] = pd.melt(data[key],id_vars=['Player','Pos','Team','Year'],\
#                           value_vars=['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17'])
#                 data[key].rename(columns={'variable':'Week','value':'Snaps'},inplace=True)
#                 data[key].replace({'Team':\
#                           {'GB':'GNB','JAC':'JAX','KC':'KAN','NE':'NWE','NO':'NOR','SF':\
#                            'SFO','TB':'TAM','Multi':None,'LV':'OAK',},'Snaps':{'bye':None}},inplace=True)
#                 data[key].dropna(thresh=3,inplace=True)

#             player = pd.concat(data.values(), ignore_index=True)
        #df.to_csv(f'nflinjuryreport_{teams[team]}_{years[year]}.csv',header=True)

        #advancing through url
        if year < len(years)-1:
            year += 1

        else:
            year = 0
            team += 1
    print(len(data))

In [ ]:
#webscrap_nfl_injury_reports(2017,2019)

In [ ]:
"""def webscrap_nfl_injury_reports(Start_year,End_year):
    #list of the NFL teams for url purposes
    teams = ['crd', 'atl', 'rav', 'buf', 'car', 'chi', 'cin', 'cle', 'dal', 'den', 'det', 'gnb','htx','clt','jax','kan',
             'sdg','ram','mia','min','nor','nwe','nyg','nyj','rai','phi','pit','sea','sfo','tam','oti','was']
    years = []

    #list of years to pull data for url purposes
    for i in range(Start_Year,End_Year):
        years.append(i)

    #starting points to iterate through
    team = 0
    year = 0
    dfname = []
    try:
        while team < 32:
            #url for web scraping
            url = f'https://www.pro-football-reference.com/teams/{teams[team]}/{years[year]}_injuries.htm'

            #opening website
            r = requests.get(url)
            soup = BeautifulSoup(r.content, 'lxml')

            #finding table
            table = soup.find('table', attrs={'class': 'sortable', 'id': 'team_injuries'})
            table_rows = table.find_all('tr')

            #scraping the data
            final_data = []
            for tr in table_rows:
                td = tr.find_all(['th','td'])
                row = [tr['data-tip'] if tr.has_attr("data-tip") else tr.text for tr in td]
                final_data.append(row)

            # update of location of web scrape
            print(teams[team]+" "+years[year])

            #creatingdataframe to save
            data = final_data[1:]
            data_body = [[data[j][i] for j in range(len(data))] for i in range(len(data[0]))]
            df = pd.DataFrame(data_body,final_data[0]).T           
            df.to_csv(f'nflinjuryreport_{teams[team]}_{years[year]}.csv',header=True)

            #advancing through url
            if year < 2:
                year += 1

            else:
                year = 0
                team += 1
    except:
        print('Error')"""